Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [3]:
url = 'http://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz


Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [6]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall()
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)
print("test_folders",test_folders)


notMNIST_large already present - Skipping extraction of notMNIST_large.tar.gz.
['notMNIST_large/A', 'notMNIST_large/B', 'notMNIST_large/C', 'notMNIST_large/D', 'notMNIST_large/E', 'notMNIST_large/F', 'notMNIST_large/G', 'notMNIST_large/H', 'notMNIST_large/I', 'notMNIST_large/J']
notMNIST_small already present - Skipping extraction of notMNIST_small.tar.gz.
['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']
test_folders ['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']


---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [7]:
from IPython.display import display, Image
display(Image(filename="notMNIST_small/A/Q0NXaWxkV29yZHMtQm9sZEl0YWxpYy50dGY=.png"))

In [8]:
import os 
os.listdir('/Users/parksoy/Desktop/GoogleDrive_Soyoung/Udacity_nanoDegree_MachineLearning/digit_recognition/notMNIST_small/B/') 

['MDEtMDEtMDAudHRm.png',
 'MDRiXzA4LnR0Zg==.png',
 'MjAwcHJvb2Ztb29uc2hpbmUgcmVtaXgudHRm.png',
 'MlJlYmVsc0RldXgtQmxhY2sub3Rm.png',
 'MlRvb24gU2hhZG93LnR0Zg==.png',
 'MlRvb24yIFNoYWRvdy50dGY=.png',
 'MTAuMTUgU2F0dXJkYXkgTmlnaHQgQlJLLnR0Zg==.png',
 'MTFTMDEgQmxhY2sgVHVlc2RheSBPZmZzZXQudHRm.png',
 'MTggSG9sZXMgQlJLLnR0Zg==.png',
 'MTh0aENlbnR1cnkudHRm.png',
 'MTIgV2FsYmF1bSBJdGFsaWMgMTMyNjMudHRm.png',
 'MTJTYXJ1WWVsbG93Rm9nLnR0Zg==.png',
 'Nng3b2N0IEFsdGVybmF0ZSBFeHRyYUxpZ2h0LnR0Zg==.png',
 'Nng3b2N0IEFsdGVybmF0ZSBSZWd1bGFyLnR0Zg==.png',
 'NXRoR3JhZGVyLnR0Zg==.png',
 'OC1iaXQgTGltaXQgTyBCUksudHRm.png',
 'OEJhbGxTY3JpcHRTQ2Fwc1NTSyBJdGFsaWMudHRm.png',
 'OTExIFBvcnNjaGEgSXRhbGljLnR0Zg==.png',
 'OXNxZ3JkIFRoaW4udHRm.png',
 'Q09ERTNYLnR0Zg==.png',
 'Q0cgT21lZ2EudHRm.png',
 'Q0NDb21pY3JhenktQm9sZEl0YWxpYy50dGY=.png',
 'Q0NTcG9va3l0b290aC1SZWd1bGFyLnR0Zg==.png',
 'Q0NXaWxkV29yZHMtQm9sZEl0YWxpYy50dGY=.png',
 'Q0sgUGluay50dGY=.png',
 'Q1FOLU1vbGVjdWxhci1EeW5hbW8tUmVndWxhci5vdGY=.png',
 'Q290cmFu

In [9]:
cellname="OEJhbGxTY3JpcHRTQ2Fwc1NTSyBJdGFsaWMudHRm.png"
display(Image(filename="notMNIST_small/B/"+cellname))

Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. 

A few images might not be readable, we'll just skip them.

In [23]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  print('dataset----\n',dataset)
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)


notMNIST_large/A.pickle already present - Skipping pickling.
notMNIST_large/B.pickle already present - Skipping pickling.
notMNIST_large/C.pickle already present - Skipping pickling.
notMNIST_large/D.pickle already present - Skipping pickling.
notMNIST_large/E.pickle already present - Skipping pickling.
notMNIST_large/F.pickle already present - Skipping pickling.
notMNIST_large/G.pickle already present - Skipping pickling.
notMNIST_large/H.pickle already present - Skipping pickling.
notMNIST_large/I.pickle already present - Skipping pickling.
notMNIST_large/J.pickle already present - Skipping pickling.


AttributeError: 'list' object has no attribute 'shape'

In [17]:
imf='/Users/parksoy/Desktop/GoogleDrive_Soyoung/Udacity_nanoDegree_MachineLearning/digit_recognition/notMNIST_small/B/OEJhbGxTY3JpcHRTQ2Fwc1NTSyBJdGFsaWMudHRm.png'

ds=np.ndarray(shape=(len(imf), 28, 28),dtype=np.float32)
print(ds.shape)
pixel_depth=255.0
idata = (ndimage.imread(imf).astype(float) -  pixel_depth / 2) / pixel_depth

print(idata.shape)

ds_test = load_letter('notMNIST_small/E', 1800)

(157, 28, 28)
(28, 28)
notMNIST_small/E
dataset----
 [[[-0.5        -0.49607843 -0.48039216 ..., -0.49215686 -0.5        -0.5       ]
  [-0.49607843 -0.48431373 -0.46078432 ..., -0.42941177 -0.44509804 -0.5       ]
  [-0.5        -0.33137256 -0.32352942 ..., -0.40196079 -0.3509804
   -0.48823529]
  ..., 
  [-0.49607843 -0.40980393 -0.19019608 ..., -0.43333334 -0.45686275 -0.5       ]
  [-0.44901961 -0.44509804 -0.30392158 ..., -0.25294119 -0.49607843 -0.5       ]
  [-0.5        -0.5        -0.48823529 ..., -0.48039216 -0.5        -0.5       ]]

 [[ 0.5         0.5         0.5        ...,  0.5         0.5         0.5       ]
  [ 0.5         0.5         0.5        ...,  0.5         0.5         0.5       ]
  [ 0.5         0.5         0.5        ...,  0.5         0.5         0.5       ]
  ..., 
  [ 0.5         0.5         0.5        ...,  0.5         0.5         0.5       ]
  [ 0.5         0.5         0.5        ...,  0.5         0.5         0.5       ]
  [ 0.5         0.5         0.5     

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

In [38]:
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle

hello=pickle.load(open("/notMNIST_small/B.pickle","rb") ) #, "rb"




IOError: [Errno 2] No such file or directory: '/notMNIST_small/C.pickle'

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `train_size` as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [44]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)

  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
  print('valid_size==',valid_size)
  print('train_size==',train_size)
  print('num_classes==',num_classes)
  print('train_size // num_classes=tsize_per_class==',tsize_per_class)
  print('------')
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):
    print('label--',label,pickle_file)
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        #print('letter_set---\n',letter_set)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        #print('letter_set after shuffle--\n',letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          #print('valid_letter---\n',valid_letter)
          valid_dataset[start_v:end_v, :, :] = valid_letter
          #print('valid_dataset---\n',valid_dataset)
          valid_labels[start_v:end_v] = label
          #print('valid_labels---',valid_labels)
          start_v += vsize_per_class
          print('start_v += vsize_per_class----',start_v)
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

valid_size== 10000
train_size== 200000
num_classes== 10
train_size // num_classes=tsize_per_class== 20000
------
label-- 0 notMNIST_large/A.pickle
start_v += vsize_per_class---- 1000
label-- 1 notMNIST_large/B.pickle
start_v += vsize_per_class---- 2000
label-- 2 notMNIST_large/C.pickle
start_v += vsize_per_class---- 3000
label-- 3 notMNIST_large/D.pickle
start_v += vsize_per_class---- 4000
label-- 4 notMNIST_large/E.pickle
start_v += vsize_per_class---- 5000
label-- 5 notMNIST_large/F.pickle
start_v += vsize_per_class---- 6000
label-- 6 notMNIST_large/G.pickle
start_v += vsize_per_class---- 7000
label-- 7 notMNIST_large/H.pickle
start_v += vsize_per_class---- 8000
label-- 8 notMNIST_large/I.pickle
start_v += vsize_per_class---- 9000
label-- 9 notMNIST_large/J.pickle
start_v += vsize_per_class---- 10000
valid_size== 0
train_size== 10000
num_classes== 10
train_size // num_classes=tsize_per_class== 1000
------
label-- 0 notMNIST_small/A.pickle
label-- 1 notMNIST_small/B.pickle
label-- 2 n

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [30]:
def randomize(dataset, labels):
  print('labels--',labels)
  print('labels.shape----',labels.shape)
  print('labels.shape[0]----',labels.shape[0])
  permutation = np.random.permutation(labels.shape[0])
  print('permutation--',permutation)
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels

train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

labels-- [0 4 6 ..., 1 8 5]
labels.shape---- (200000,)
labels.shape[0]---- 200000
permutation-- [ 53113 157753 113760 ..., 161425  58170  47895]
labels-- [4 8 8 ..., 7 3 7]
labels.shape---- (10000,)
labels.shape[0]---- 10000
permutation-- [7052 8386 5114 ..., 7963 9985 3178]
labels-- [8 3 8 ..., 9 7 3]
labels.shape---- (10000,)
labels.shape[0]---- 10000
permutation-- [3592 5867  794 ..., 5769 8145 8461]


In [36]:
print(valid_dataset.shape)
print(valid_labels.shape)
print(valid_labels)

(10000, 28, 28)
(10000,)
[2 7 0 ..., 4 2 5]


---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

---

Finally, let's save the data for later reuse:

In [7]:
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [29]:
pickle_file = 'notMNIST.pickle'
statinfo = os.stat(pickle_file)
print('os.stat(pickle_file)--\n',os.stat(pickle_file))
print('Compressed pickle size:', statinfo.st_size)

os.stat(pickle_file)--
 posix.stat_result(st_mode=33188, st_ino=46963148, st_dev=16777220, st_nlink=1, st_uid=501, st_gid=20, st_size=690800441, st_atime=1485472775, st_mtime=1484921429, st_ctime=1484921429)
Compressed pickle size: 690800441


In [43]:
with open(pickle_file, 'rb') as f:
        ds_total = pickle.load(f)
print(ds_total)

{'train_labels': array([4, 9, 6, ..., 2, 4, 4], dtype=int32), 'valid_labels': array([1, 9, 3, ..., 8, 9, 8], dtype=int32), 'test_labels': array([3, 6, 4, ..., 6, 9, 0], dtype=int32), 'test_dataset': array([[[-0.5       , -0.5       , -0.5       , ..., -0.48823529,
         -0.5       , -0.5       ],
        [-0.5       , -0.5       , -0.5       , ..., -0.5       ,
         -0.48823529, -0.5       ],
        [-0.5       , -0.5       , -0.5       , ...,  0.02941176,
         -0.5       , -0.49215686],
        ..., 
        [-0.5       , -0.48823529, -0.5       , ..., -0.5       ,
         -0.5       , -0.5       ],
        [-0.18235295,  0.13529412,  0.07254902, ..., -0.5       ,
         -0.5       , -0.5       ],
        [ 0.22941177,  0.5       ,  0.49215686, ..., -0.5       ,
         -0.5       , -0.5       ]],

       [[-0.5       , -0.5       , -0.5       , ..., -0.46470588,
         -0.5       , -0.49607843],
        [-0.5       , -0.5       , -0.5       , ..., -0.25294119,
     

In [58]:
print(type(ds_total))
#print(size(ds_total))
print(ds_total['train_labels'])

<type 'dict'>
[4 9 6 ..., 2 4 4]


---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---
Overall discussion:
https://discussions.udacity.com/t/assignment-1-train-dataset-and-valid-dataset-overlap-question/45193/9

MSE
http://www.pyimagesearch.com/2014/09/15/python-compare-two-images/

---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [55]:
import sklearn.linear_model
from sklearn import linear_model


In [68]:
print(type(ds_total['train_dataset']))
print(ds_total['train_dataset'].shape)
print(ds_total['train_labels'].shape)
print(ds_total['test_dataset'].shape)
print(ds_total['test_labels'].shape)
print(ds_total['train_dataset'])


<type 'numpy.ndarray'>
(200000, 28, 28)
(200000,)
(10000, 28, 28)
(10000,)
[[[-0.5        -0.49607843 -0.5        ..., -0.5        -0.49215686 -0.5       ]
  [-0.49607843 -0.47647059 -0.5        ..., -0.5        -0.47254902
   -0.49607843]
  [-0.49607843 -0.49607843 -0.5        ..., -0.5        -0.49607843
   -0.49607843]
  ..., 
  [-0.49607843 -0.49215686 -0.5        ..., -0.5        -0.49215686
   -0.49607843]
  [-0.49607843 -0.47647059 -0.5        ..., -0.5        -0.47254902
   -0.49607843]
  [-0.5        -0.49607843 -0.5        ..., -0.5        -0.49607843 -0.5       ]]

 [[-0.5        -0.5        -0.5        ...,  0.48823529  0.5         0.1509804 ]
  [-0.5        -0.5        -0.5        ...,  0.48431373  0.14705883
   -0.32745099]
  [-0.5        -0.5        -0.5        ..., -0.32745099 -0.5        -0.49607843]
  ..., 
  [-0.5        -0.44901961  0.1509804  ..., -0.5        -0.5        -0.5       ]
  [-0.49607843 -0.49607843 -0.49215686 ..., -0.5        -0.5        -0.5       ]
 

In [76]:
print(ds_total['train_dataset'].shape)
trans_train_dataset=ds_total['train_dataset'].reshape(200000,-1) #transpose(2,0,1)
print(trans.shape)
print(ds_total['train_dataset'])
print('-----')
print(trans)


(200000, 28, 28)
(200000, 784)
[[[-0.5        -0.49607843 -0.5        ..., -0.5        -0.49215686 -0.5       ]
  [-0.49607843 -0.47647059 -0.5        ..., -0.5        -0.47254902
   -0.49607843]
  [-0.49607843 -0.49607843 -0.5        ..., -0.5        -0.49607843
   -0.49607843]
  ..., 
  [-0.49607843 -0.49215686 -0.5        ..., -0.5        -0.49215686
   -0.49607843]
  [-0.49607843 -0.47647059 -0.5        ..., -0.5        -0.47254902
   -0.49607843]
  [-0.5        -0.49607843 -0.5        ..., -0.5        -0.49607843 -0.5       ]]

 [[-0.5        -0.5        -0.5        ...,  0.48823529  0.5         0.1509804 ]
  [-0.5        -0.5        -0.5        ...,  0.48431373  0.14705883
   -0.32745099]
  [-0.5        -0.5        -0.5        ..., -0.32745099 -0.5        -0.49607843]
  ..., 
  [-0.5        -0.44901961  0.1509804  ..., -0.5        -0.5        -0.5       ]
  [-0.49607843 -0.49607843 -0.49215686 ..., -0.5        -0.5        -0.5       ]
  [-0.5        -0.49607843 -0.48823529 ..., -

In [77]:
logistic = linear_model.LogisticRegression()
trans_train_dataset=ds_total['train_dataset'].reshape(200000,-1)
trans_test_dataset=ds_total['test_dataset'].reshape(10000,-1)

print('LogisticRegression score: %f'
      % logistic.fit(trans_train_dataset, ds_total['train_labels']).score(trans_test_dataset, ds_total['test_labels']))


LogisticRegression score: 0.893300


In [78]:
print('hi')

hi
